**Draft Model for Mice Steep Stage Analysis**

In [99]:
import random
import numpy as np
import pandas as pd
import os
import re
import copy
import matplotlib.pyplot as plt

import torch
from torch import optim, nn
from torchvision import transforms, datasets, models

from collections import OrderedDict

import torch.utils.data as utils

from sklearn.preprocessing import LabelBinarizer

import tkinter as tk #tk for file dialog (requires Jinja2!!!)
from tkinter import filedialog #tkinter for file dialog

import re #regex for parsing
from os.path import exists
from inputMassager import *

Now we'd import the data and create a dataframe out of it. (SHOUTOUT REGAN!!)

In [100]:
inputHandler = inputMassager()

filepath = inputHandler.askForInput("Blah!")

#Period Size Variable : effects CNN architecture
periodSize = 200

#makePeriodFromTxt(self, filepath, periodSize, maxPeriods=None):
df = inputHandler.makePeriodFromTxt(filepath, periodSize, 2000)

# target = regan and audreys code ---> This is what will go into the data loader


In [101]:
filepath2 = inputHandler.askForInput("Blah!")
timestamps = find_time_labels(df, filepath2)
labels = label_dataframe(df, timestamps)

labels
"""
figure out how to integrate this

# set up training and validation data
train_data, train_labels, val_data, val_labels = # USE DATAFRAME TO MAKE THIS

train = 
train_loader = utils.DataLoader(train, batch_size=64, shuffle=True)

test = 
test_loader = utils.DataLoader(test, batch_size=64, shuffle=True)

data_loaders = {'train': train_loader, 'valid': test_loader}
dataset_sizes = {'train': len(train), 'valid': len(test)}
"""


time we're on: 1703.0699462890625
my time is 1, current epoch is (1.004,2.004)
time we're on: 1707.9849853515625
my time is 2, current epoch is (1703.441,1704.441)
time we're on: 1721.60498046875
my time is 3, current epoch is (1708.466,1709.466)
time we're on: 1728.739990234375
my time is 4, current epoch is (1722.536,1723.536)
time we're on: 1732.344970703125
my time is 5, current epoch is (1729.571,1730.571)
time we're on: 1743.1099853515625
my time is 6, current epoch is (1732.586,1733.586)
time we're on: 2087.1650390625
my time is 7, current epoch is (1743.641,1744.641)


"\nfigure out how to integrate this\n\n# set up training and validation data\ntrain_data, train_labels, val_data, val_labels = # USE DATAFRAME TO MAKE THIS\n\ntrain = \ntrain_loader = utils.DataLoader(train, batch_size=64, shuffle=True)\n\ntest = \ntest_loader = utils.DataLoader(test, batch_size=64, shuffle=True)\n\ndata_loaders = {'train': train_loader, 'valid': test_loader}\ndataset_sizes = {'train': len(train), 'valid': len(test)}\n"

In [102]:
df['labels'] = labels

print(df.head())


train_loader = torch.utils.data.DataLoader(df, batch_size=64, shuffle=False)
print(train_loader)

   start    end                                                 c1   
0  0.000  0.999  [-0.04, 0.23, 0.2, 0.29, 0.23, 0.22, 0.25, 0.2...  \
1  1.004  2.004  [0.0, 0.13, 0.03, -0.01, -0.02, -0.02, -0.02, ...   
2  2.009  3.009  [-0.02, -0.08, -0.18, -0.13, -0.05, -0.08, -0....   
3  3.014  4.014  [0.06, 0.04, 0.04, 0.06, 0.08, 0.06, 0.03, 0.0...   
4  4.019  5.019  [0.03, 0.11, 0.22, 0.11, 0.11, 0.02, 0.06, 0.0...   

                                                  c2  labels  
0  [0.2, -0.14, -0.63, -0.44, 0.13, 0.0, 0.2, -0....       4  
1  [0.09, 0.04, 0.61, 0.83, 0.16, 0.12, 0.31, 0.6...       2  
2  [-0.15, -0.02, 0.16, -0.26, -0.35, 0.18, -0.19...       2  
3  [-0.14, 0.16, -0.12, 0.03, -0.11, 0.08, -0.03,...       2  
4  [0.37, -0.21, 0.25, -0.3, -0.16, 0.24, -0.58, ...       2  


In [103]:
# classes for classification tasks (what sleep stage the mouse is in)
# based on labels from annotated data, W is wake, N is Non-REM, R is REM, and A is artifact (unique to our model)

classes = {0: "W", 1: "N", 2: "R", 3: "A"}

**Creating class for Model**

In [104]:
""" the left hand side of the CNN"""
def CNN_eeg_layer1(fs): 
    return nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=fs//2, stride=2, padding=2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.MaxPool1d(kernel_size=8, stride=8),
            
            nn.Conv1d(64, 64, kernel_size=8, padding=2),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=8, padding=2),
            nn.ReLU(), 
            nn.Conv1d(64, 64, kernel_size=8, padding=2),
            nn.ReLU())


class CNN(nn.Module):
  
    def __init__(self, period_size, num_classes=4):
      
        super(CNN, self).__init__()
        self.period_size = period_size
        self.conv1 = nn.Conv1d(in_channels=period_size, out_channels=64, kernel_size=period_size//2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=period_size//2)
        self.conv3 = nn.Conv1d(in_channels=period_size, out_channels=64, kernel_size=period_size//2)
        self.conv4 = nn.Conv1d(in_channels=period_size, out_channels=1048, kernel_size=period_size//2)
        self.fc1 = nn.Linear(1048, 50)
        self.fc2 = nn.Linear(50, 4)
        self.dropout = nn.Dropout(0.3)
        
                
        
    def forward(self, channels):
      
        # at some point, we'll have a second channel for emg
        
        x = channels # extract eeg channel (Channel 1) from data frame
        # ch2 = # extract emg channel (Channel 2) from data frame
        
        x = self.conv1(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size=2)

        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size=2)

        x = self.conv3(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size=2)

        x = self.conv4(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size=2)

        # 4 layers in total
        x = x.view(-1, 128*2*2)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)

        x = self.dropout(x)
        
        x = self.fc2(x)
        x = torch.nn.functional.log_softmax(x, dim=1)
        return x

        return out

**Training the Model**

We can play around with the parameters a bit. I've included the function for counting the number of parameters from the 06-convnet.ipynb from the CSE144 example repo.

For our optimization, I used Adam because it converges faster and I don't think our data is well-formatted enough yet to get decent results with SGD.

In [105]:
# function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

In [106]:
# make our model
model = CNN(period_size=200)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# define optimization function and print number of params
optimizer = optim.Adam(model.parameters(), lr=0.005)
print('Number of parameters: {}'.format(get_n_params(model)))

Number of parameters: 23989894


In one example I saw, they used [model.state_dict()](https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html) to record the best learnable parameters (i.e. weights and biases) of a model. They use it for storing the best possible model found during training.

In [107]:
def train_model(model, optimizer, scheduler, num_epochs):
    
    # deep copy and save the best model weights found
    best_model_wts = copy.deepcopy(model.state_dict())
    # initialize best accuracy found to 0
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        model.train()
                
        running_loss = 0.0
        running_corrects = 0

        for index, (inputs, labels) in enumerate(zip(train_loader.dataset['c1'], train_loader.dataset['c2'])):
            print(inputs)
            inputs = torch.tensor(inputs).to(device)
            print(inputs.size)
            labels = torch.tensor(labels).to(device)

            optimizer.zero_grad()

            # only compute gradients during training, not
            # necessary in validations
            outputs = model(torch.tensor(inputs).view(1, 200))
            _, preds = torch.max(outputs, 1)
            loss = torch.nn.functional.nll_loss(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(train_loader.dataset)
            epoch_acc = running_corrects.double() / len(train_loader.dataset)

            print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()
    print('Best Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [108]:

sched = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
train_model(model, optimizer,sched, 10)

Epoch 1/10
[-0.04, 0.23, 0.2, 0.29, 0.23, 0.22, 0.25, 0.21, 0.24, 0.21, 0.15, 0.22, 0.16, 0.14, 0.19, 0.11, 0.14, 0.17, 0.19, 0.16, 0.19, 0.21, 0.22, 0.28, 0.23, 0.23, 0.29, 0.22, 0.14, 0.12, 0.06, 0.15, 0.2, 0.16, 0.16, 0.14, 0.05, 0.08, 0.08, 0.1, 0.09, 0.06, 0.1, 0.11, 0.08, 0.0, 0.04, 0.04, 0.09, 0.12, 0.17, 0.18, 0.17, 0.19, 0.2, 0.2, 0.22, 0.17, 0.15, 0.02, 0.01, 0.04, 0.04, 0.06, 0.04, 0.02, -0.0, 0.01, 0.03, 0.03, -0.0, 0.08, 0.05, 0.04, 0.03, -0.01, 0.04, 0.1, 0.12, 0.09, 0.13, 0.14, 0.09, 0.11, 0.01, -0.07, -0.05, -0.05, -0.01, -0.02, -0.1, -0.07, -0.09, -0.08, -0.09, -0.11, -0.11, -0.07, -0.09, -0.06, -0.04, -0.05, -0.1, -0.07, -0.13, -0.1, -0.11, -0.09, -0.03, -0.05, -0.08, -0.05, -0.14, -0.06, -0.12, -0.14, -0.17, -0.14, -0.15, -0.07, -0.09, -0.04, 0.02, -0.02, 0.04, 0.02, -0.01, -0.03, 0.01, 0.01, 0.01, -0.02, -0.08, -0.09, -0.09, -0.11, -0.09, -0.09, -0.11, -0.04, -0.09, -0.05, -0.07, -0.05, -0.08, -0.02, -0.01, -0.05, -0.13, 0.04, -0.04, 0.0, -0.08, -0.07, -0.07, -0.1, 

/var/folders/dg/6gzrkj09311g91kn90n1qdcc0000gn/T/ipykernel_1860/1889929946.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(inputs).view(1, 200))


UnboundLocalError: local variable 'x' referenced before assignment